[View in Colaboratory](https://colab.research.google.com/github/kyokicchi/kaggle_codes/blob/master/predict.ipynb)

In [1]:
!pip install lightgbm

    100% |████████████████████████████████| 1.1MB 17.8MB/s 


In [2]:
!pip install --upgrade xgboost

    100% |████████████████████████████████| 15.8MB 2.8MB/s 
  Found existing installation: xgboost 0.7.post4
    Uninstalling xgboost-0.7.post4:
      Successfully uninstalled xgboost-0.7.post4


In [3]:
from google.colab import drive
from google.colab import files

import pickle

import numpy as np
import pandas as pd

import xgboost as xgb
import lightgbm as lgb
from keras import models
from keras import layers
from keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.metrics import mean_squared_error


Using TensorFlow backend.


In [4]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
%cd ../content/gdrive/My Drive/projects/

/content/gdrive/My Drive/projects


In [6]:
%ls

model_LGB.sav  model_XGB.sav  user_test_n.csv
model_NN.h5    output.csv     user_train_n.csv


In [7]:
%%time
df_user_train = pd.read_csv('user_train_n.csv', index_col = 0, dtype={'fullVisitorId': 'str'})
df_user_test = pd.read_csv('user_test_n.csv', index_col = 0, dtype={'fullVisitorId': 'str'})

df_y = df_user_train['transactionRevenue_sum']
df_x = df_user_train.drop(['transactionRevenue_sum'], axis=1)
df_tgt = df_user_test.drop(['transactionRevenue_sum'], axis=1)

x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, random_state=1)
x_eval, x_valid, y_eval, y_valid = train_test_split(x_test, y_test, random_state=1)

CPU times: user 10.8 s, sys: 1.27 s, total: 12.1 s
Wall time: 16.2 s


In [11]:
%%time
models = []

filename = 'model_LGB.sav'
model_LGB = pickle.load(open(filename, 'rb'))
models.append(model_LGB)

filename = 'model_XGB.sav'
model_XGB = pickle.load(open(filename, 'rb'))
models.append(model_XGB)

filename = 'model_NN.h5'
model_NN = load_model(filename)
models.append(model_NN)



CPU times: user 688 ms, sys: 150 ms, total: 838 ms
Wall time: 3.57 s


In [12]:
for model in models:
    y_pred = model.predict(x_valid.fillna(0))
    mse = mean_squared_error(y_valid, y_pred)
    rmse = np.sqrt(mse)
    print(rmse)

42483661.47583807
39313308.87103267
49815171.8607874


In [13]:
%%time
preds = []
for model in models:
    pred_tmp = model.predict(df_tgt)
    preds.append(pred_tmp)

CPU times: user 28.2 s, sys: 2.68 s, total: 30.9 s
Wall time: 22.1 s


In [14]:
%%time
df_preds = pd.DataFrame(preds).T

CPU times: user 33.4 s, sys: 389 ms, total: 33.8 s
Wall time: 33.8 s


In [15]:
df_preds.head()

,0,1,2
0,2.82722e+06,989150,[5986.08]
1,328303,438966,[6021.888]
2,328303,438966,[3495.1853]
3,328303,438966,[5265.9126]
4,567555,438966,[3837.3691]


In [0]:
pred_tgt = df_preds[df_preds.columns].sum(axis=1)/len(preds)
pred_tgt[pred_tgt<0] = 0
pred_tgt = pred_tgt.values.tolist()

In [17]:
pred_tgt[:5]

[1274117.25,
 257763.58333333334,
 256921.35416666666,
 257511.60416666666,
 336785.9583333333]

In [0]:
df_tgt['PredictedLogRevenue'] = pred_tgt

In [0]:
df_tgt[['PredictedLogRevenue']].to_csv("output.csv", index=True)

In [21]:
df_output = pd.read_csv('output.csv')
df_output.head()

,fullVisitorId,PredictedLogRevenue
0,2.596787e+11,1.274117e+06
1,3.038793e+12,2.577636e+05
2,9.663019e+12,2.569214e+05
3,1.043846e+13,2.575116e+05
4,1.099298e+13,3.367860e+05


In [24]:
type(df_output['fullVisitorId'][0])

numpy.float64